<div>
<a href="http://bokeh.pydata.org/"><img src="../images/bokeh-header.png"></a>
</div>

# Bokeh 快速概览

Bokeh 是一个Python交互式可视化库，用于在现代Web浏览器上进行展示。它的目标是以D3.js风格提供优雅简洁的图形展示，并与大数据或流数据进行高效交互。Bokeh可以帮助快速轻松地创建交互图，仪表板，和数据应用。

## 简单示例

这是一个简单的例子。首先，我们从 [`bokeh.plotting`](https://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html) 导入 [`figure`](https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.figure) 函数，从而让我们轻松创建各种有趣的图表。我们还从 `bokeh.io` 导入 `show` 和 `ouptut_notebook` 函数 &mdash; 这使得我们能在Notebook中以内联的形式显示结果。

In [1]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show

接下来，我们会告诉 Bokeh 直接在 Notebook 中显示图表。
这将导致所有JavaScript和数据直接嵌入到Notebook的HTML中。
（Bokeh可以直接输出到HTML文件，或者启动一个web server，我们后面会看到）

In [2]:
output_notebook()

Loading BokehJS ...

接着，我们导入NumPy并且创建一些简单数据。

In [3]:
from numpy import cos, linspace
x = linspace(-6, 6, 100)
y = cos(x)

现在，我们调用Bokeh的`figure`函数创建一个图表`p`。然后，我们调用其`circle()`方法在每个X,Y点处渲染一个红圈。

我们可以立即操作该图表：

  * 点击-拖动 可以平移图表
  * 鼠标滚轮 可以放大和缩小（先在工具栏选取）
  
下图中的工具栏是所有图表都可用的默认工具栏。它可以通过`tools`关键字参数进一步配置。

In [4]:
p = figure(width=500, height=500)
p.circle(x, y, size=7, color="firebrick", alpha=0.5)
show(p)

# 柱状图示例


Bokeh的核心显示模型依赖于绑定到数据序列的 *图形原语*。这类似于Protovis和D3，和其他大多数Python绘图库不同。

一个稍微复杂一些的例子演示了这个想法。

Bokeh在`bokeh.sampledata`包内置了一些有趣的数据集 "sample data" 。我们会加载一些汽车里程历史数据，数据类型为Pandas的`DataFrame`。

In [5]:
from bokeh.sampledata.autompg import autompg

grouped = autompg.groupby("yr")
mpg = grouped["mpg"]
avg = mpg.mean()
std = mpg.std()
years = list(grouped.groups.keys())
american = autompg[autompg["origin"]==1]
japanese = autompg[autompg["origin"]==3]

我们要绘制每年的英里数分布图。

In [6]:
p = figure(title="MPG by Year (Japan and US)")

p.vbar(x=years, bottom=avg-std, top=avg+std, width=0.8, 
       fill_alpha=0.2, line_color=None, legend="MPG 1 stddev")

p.circle(x=japanese["yr"], y=japanese["mpg"], size=10, alpha=0.5,
         color="red", legend="Japanese")

p.triangle(x=american["yr"], y=american["mpg"], size=10, alpha=0.3,
           color="blue", legend="American")

p.legend.location = "top_left"
show(p)

D:\ProgramData\Anaconda3\lib\site-packages\bokeh\core\json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


### 这种方法可以用来生成其它有趣的图表。这里有更多例子 [Bokeh Documentation Gallery](https://bokeh.pydata.org/en/latest/docs/gallery.html)。

## Linked Brushing

为了在一个数据层面上关联多个图表，我们可以显式地将数据放置于`ColumnDataSource`。这使得我们可以按名称引用数据列。

我们可以使用"select"工具在图上选择点，然后其它图上的关联点将高亮显示。

In [7]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

source = ColumnDataSource(autompg)

options = dict(plot_width=300, plot_height=300,
               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select")

p1 = figure(title="MPG by Year", **options)
p1.circle("yr", "mpg", color="blue", source=source)

p2 = figure(title="HP vs. Displacement", **options)
p2.circle("hp", "displ", color="green", source=source)

p3 = figure(title="MPG vs. Displacement", **options)
p3.circle("mpg", "displ", size="cyl", line_color="red", fill_color=None, source=source)

p = gridplot([[ p1, p2, p3]], toolbar_location="right")

show(p)

你可以在 [Providing Data for Plots and Tables](https://bokeh.pydata.org/en/latest/docs/user_guide/data.html) 了解更多关于`ColumnDataSource`和其它Bokeh数据结构。

## 独立 HTML

除了和 Notebook 协同工作，Bokeh 也可以把图表保存到 HTML 文件里。这个是前面的条形图示例，但是保存到独立的文件中。

现在，当我们调用`show()`时，一个新的浏览器页面会打开图表。如果我们只是想保存文件，我们可以使用`save()`。

In [8]:
from bokeh.plotting import output_file

output_file("barplot.html")

p = figure(title="MPG by Year (Japan and US)")

p.vbar(x=years, bottom=avg-std, top=avg+std, width=0.8, 
       fill_alpha=0.2, line_color=None, legend="MPG 1 stddev")

p.circle(x=japanese["yr"], y=japanese["mpg"], size=10, alpha=0.3,
         color="red", legend="Japanese")

p.triangle(x=american["yr"], y=american["mpg"], size=10, alpha=0.3,
           color="blue", legend="American")

p.legend.location = "top_left"
show(p)

D:\ProgramData\Anaconda3\lib\site-packages\bokeh\core\json_encoder.py:80: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


## Bokeh 应用

Bokeh 还有一个 server 模块，用来创建交互式Web应用，可以轻松地把强大的pydata系列工具和精致的Bokeh可视效果连接起来。Bokeh server 可用于：

* 响应浏览器中生成的UI和工具事件，借助Python的全部功能进行计算或查询
* 自动将服务器端更新推送给UI（即浏览器中的小部件或图表）
* 使用周期，超时，和异步回调函数来流动更新

点击下面的缩略图来启动Bokeh应用。

<center>
<a href="https://demo.bokehplots.com/apps/crossfilter">
  <img 
    width="30%" height="30%" style="display: inline ; padding: 10px;" 
    src="https://bokeh.pydata.org/static/crossfilter_t.png"
  >
</a>

<a href="https://demo.bokehplots.com/apps/movies">
  <img 
    width="30%" height="30%" style="display: inline ; padding: 10px;" 
    src="https://bokeh.pydata.org/static/movies_t.png" 
  >
</a>

<a href="https://demo.bokehplots.com/apps/gapminder">
  <img 
    width="30%" height="30%" style="display: inline ; padding: 10px;" 
    src="http://bokeh.pydata.org/static/gapminder_t.png"
  >
</a>
</center>

关于开发和部署Bokeh server应用的更多的细节和信息，请参考用户指南章节 [Running a Bokeh Server](https://bokeh.pydata.org/en/latest/docs/user_guide/server.html)

## BokehJS

Bokeh 的核心，由一个JavaScript库 [BokehJS](https://github.com/bokeh/bokeh/tree/master/bokehjs)，和一个 Python binding 组成。该 Python binding 提供类和对象以最终生成图表结构的JSON表示。

您可以在用户参考指南的 [Developing with JavaScript](https://bokeh.pydata.org/en/latest/docs/user_guide/bokehjs.html) 章节找到更多的设计和使用资料。

## 更多信息

在下面列出的资源中，可找到更多的细节和信息：

*Documentation:* https://bokeh.pydata.org/en/latest

*GitHub:* https://github.com/bokeh/bokeh

*Mailing list:* [bokeh@anaconda.com](mailto:bokeh@anaconda.com)

*Gitter Chat:* https://gitter.im/bokeh/bokeh

Be sure to follow us on Twitter [@bokehplots](http://twitter.com/BokehPlots>) and on [Youtube](https://www.youtube.com/c/Bokehplots)!

<img src="../images/bokeh-transparent.png" width="64px" height="64px">